In [1]:
from astropy.io import fits
from astropy.table import Table, Column, join, hstack, vstack, unique, setdiff
from astropy import units as u
from astropy.coordinates import SkyCoord, match_coordinates_sky, Angle
from astropy.time import Time
#import desitarget.io as io
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob
import sys
import healpy as hp
import glob
import json
import lcdata
import get_redshift
import time
import h5py
#import parsnip


import sqlite3
import os
import logging

In [3]:
# path = '/global/cfs/cdirs/desi/science/td/DECam/imaging/LCData_Legacy'
path = 'LCData_Legacy'

In [4]:
class lc_data_container():
    
    def __init__(self, servey, container=[]):
        self.servey = servey
        self.container = container
        
        #dataset = lcdata.from_light_curves([light_curve])
        
    add_lc = lambda self, x: self.container.append(x)
    
    
class transients():
    filter_type = 'undeclared'
    light_curve = None
    meta = None
    
    def __init__(self, data):
        self.data = data
        
    def assign_meta(self, meta_data):
        self.meta = meta_data
        self.ra_pseudo_host = meta_data['RA-PSEUDO-HOST']
        self.dec_pseudo_host = meta_data['DEC-PSEUDO-HOST']
    
    def use_lsst_filters(self):
        if self.filter_type == 'undeclared':
            self.data['band'] = self.data['band'].replace(['r','g','z'],['lsstr','lsstg', 'lsstz'])
        else: self.data['band'] = self.data['band'].replace(['desr','desg', 'desz'],['lsstr','lsstg', 'lsstz'])
        self.filter_type = 'lsst'    
    
    def use_des_filters(self):
        if self.filter_type == 'undeclared':
            self.data['band'] = self.data['band'].replace(['r','g','z'],['desr','desg', 'desz'])
        else: self.data['band'] = self.data['band'].replace(['lsstr','lsstg', 'lsstz'],['desr','desg', 'desz'])
        self.filter_type = 'des'
        
        # flux = 10**((22.5-mag)/2.5)
        # fluxerr = flux * np.log(10) / 2.5 * magerr
    def create_lc_data(self):
        flux = list(10**((22.5-self.data['mag'])/2.5))
        magerr = list(self.data['magerr'])
        fluxerr = np.array(flux) * np.log(10) / 2.5 * np.array(magerr)
        
        self.light_curve = Table({
        'time': list(self.data['time']),
        'flux': flux,
        'fluxerr': list(fluxerr),
        'band': list(self.data['band']),})
        
        self.light_curve.meta = {
        'object_id': self.meta['ObjectID'],
        'ra': self.meta['RA-OBJECT'],
        'dec': self.meta['DEC-OBJECT'],
        'type': 'Unknown',
        'redshift': self.meta['redshift'],
        'hostgal_photoz': self.meta['hostgal_photoz'],
        'hostgal_photoz_err': self.meta['hostgal_photoz_err']}
        
        

In [5]:
transient_dirs = []
legacy = lc_data_container('LCData_Legacy')
for dirname, dirnames, filenames in os.walk(path):
    for subdirname in dirnames:
        transient_dirs.append(os.path.join(dirname, subdirname))

for dir in transient_dirs:
    
    for file in glob.glob(dir+'/lc*.csv'):
        data = pd.read_csv(file)
        data.rename(columns = {'filter':'band'}, inplace = True)
        data.rename(columns = {'mjd':'time'}, inplace = True)
        alert_mask=data['alert']
        masked_data=data[alert_mask]
        masked_data.drop(columns='alert')
        transient = transients(masked_data)
    for file in glob.glob(dir+'/object-summary.json'):
        with open(file) as f:
            meta_data = json.load(f)
            
            transient.assign_meta(dict(meta_data))
            
    legacy.add_lc(transient)



In [6]:
first = legacy.container[0]
print(first.meta.keys())
start_tot = time.time()
for i, j in enumerate(legacy.container[:10]):
    start = time.time()
    a = get_redshift.do_all(j.ra_pseudo_host, j.dec_pseudo_host)
    end = time.time()
    print('runtime:', end - start)
    if a['z_spec'] != -99:
        j.meta['redshift'] = a['z_spec']
        j.meta['z_spec'] = 1
    else:
        j.meta['redshift'] = a['mean']
        j.meta['z_spec'] = 0
    j.meta['hostgal_photoz'] = a['mean']
    j.meta['hostgal_photoz_err'] = a['err']
    print(a)
end_tot = time.time()
print('total runtime:', end_tot - start_tot)

dict_keys(['ObjectID', 'RA-OBJECT', 'DEC-OBJECT', 'NumberAlerts', 'MaxSCORE', 'RA-PSEUDO-HOST', 'DEC-PSEUDO-HOST', 'SEP-PSEUDO-HOST', 'RA-NEIGHBOR-STAR', 'DEC-NEIGHBOR-STAR', 'SEP-NEIGHBOR-STAR', 'Discovery-Round', 'Discovery-Time', 'Discovery-Filter', 'Discovery-Magnitude', 'Discovery-SNR', 'Latest-Round', 'Latest-Time', 'Latest-Filter', 'Latest-Magnitude', 'Latest-SNR'])


OSError: File-like object does not have a 'write' method, required for mode 'ostream'.

In [ ]:
for i in legacy.container[:10]:
    i.use_lsst_filters()
    
for i in legacy.container[:20]:
    print(i.filter_type)

In [ ]:
for i in legacy.container[:10]:
    i.create_lc_data()
    print(i.light_curve.meta)

In [ ]:
dataset = lcdata.from_light_curves([i.light_curve for i in legacy.container[:10]])
dataset.write_hdf5('./dataset.h5')


In [ ]:
dataset = parsnip.load_dataset('./dataset.h5')

In [ ]:
dataset.meta

In [ ]:
dataset.light_curves

In [ ]:
a = legacy.container[561]
print(a)
print(a.meta)

In [ ]:
with h5py.File('dataset.h5', 'r') as f:
    print(f.keys())
    obs = f['observations']